# 5章 誤差逆伝播法

- 重みパラメータに関する損失関数の勾配は数値微分によって求めた
- 実装は簡単だけど、時間がかかる難点がある
- 重みパラメータの勾配計算を効率良く行う「誤差逆伝播法」を学ぶ
- 誤差逆伝播法の理解には数式と計算グラフが良さそう（筆者的に）
- この章では計算グラフで説明する
- 計算グラフによって説明するアイデアは↓を参考にしてるらしい
- http://karpathy.github.io/neuralnets/
- http://cs231n.stanford.edu/


> ここで、たとえば、リンゴの値段が値上がりした場合、最終的な支払金額にどのように影響するかを知りたいとしましょう。これは、「リンゴの値段に関する支払金額の微分」を求めることに相当します。

In [1]:
from sympy import *

z = Function("z")
t = Symbol("t")
x = Symbol("x")
y = Symbol("y")
z = (x + y) ** 2
t = (x + y)


In [2]:
diff(z, x)

2*x + 2*y

In [3]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy


In [4]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print(price)

# backward

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)


220.00000000000003
2.2 110.00000000000001 200


次回

## 5.4.2 加算レイヤの実装

デュエルスタンバイ

In [5]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    


In [6]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1


# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num) 
orange_price = mul_orange_layer.forward(orange, orange_num) 
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [3]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx


In [8]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [9]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx


次回

## 5.6 Affine / Softmaxレイヤの実装

デュエルスタンバイ

In [10]:
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

Y = np.dot(X, W) + B

In [11]:
Y

array([ 0.66314273,  1.34595781,  0.56241065])

行列の内積の微分がなんで転置行列になるかわからん
http://www.ms.hum.titech.ac.jp/mayekawadocs/mat10.pdf  
http://www.r.dl.itc.u-tokyo.ac.jp/~nakagawa/SML1/math1.pdf

d (AB) / dA = B + A * dB / dA  
d (AB) / dB = dA / dB * B + A  

```
T = XW
Y = T + B
L = Y
dY / dT = 1
dT / dX
dL / dY * dY / dT * dT / dX
= dL / dY * dXW / dX
= dL / dY * dX / dX * W + dL / dY * X * dW / dX
= dL / dY * W
```

おのずとと言われていてつらめ

In [12]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])


In [13]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [14]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [18]:
dY = np.array([[1,2,3,], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [19]:
dB = np.sum(dY, axis= 0)
dB

array([5, 7, 9])

In [1]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward (self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        return dx


In [2]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

## 5.7 誤差逆伝播法

- 学習手順
  - 前提
    - 重みとバイアスがある
    - これらを訓練データに適応するように調整することを学習と呼ぶ
  - ステップ1
    - ミニバッチ (訓練データからランダムに一部データを選び出す)
  - ステップ2
    - 勾配の算出(各重みパラメータに関する損失関数の勾配を求める)
  - ステップ3
    - パラメータの更新(重みパラメータを勾配方向に微小量だけ更新する)
  - ステップ4
    - 1~3を繰り返す

誤差逆伝播法が出るのはステップ2  
数値微分は楽だけど計算時間が多いため別の方法としてこれ  

次回

### 5.7.2 誤差逆伝播法に対応したニューラルネットワークの実装

デュエルスタンバイ

In [15]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = ap.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda X: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        return grads

In [17]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist 


# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]
grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 各重みの絶対誤差の平均を求める
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))



/Users/a-tanaka/Documents/Private/Ghq/my.github.com/ara-ta3/reading-deep-learning-with-python/common/functions.py:56: RuntimeWarning: invalid value encountered in log
  return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size


AttributeError: 'NoneType' object has no attribute 'shape'

# まとめ

エラーツラミ・・・  

レイヤのモジュルー化によって自由に組み合わせることが出来るようになって、自由にネットワークを簡単に作れるらしい  
